# [Django Q](https://django-q.readthedocs.io/en/latest/)
> Django Q 是一个django任务队列, 调度器，woker使用的是python的多进程.


## [准备](https://django-q.readthedocs.io/en/latest/install.html)

- ```pip install django-q```

- ```INSTALLED_APPS = (
    'django_q',
)```
- ```python manage.py migrate```
- 选择一个消息中间件,这里用的是django自带的数据库.```Q_CLUSTER = {
    'name': 'DjangORM',
    'workers': 4,
    'timeout': 90,
    'retry': 120,
    'queue_limit': 50,
    'bulk': 10,
    'orm': 'default'
}```
- 启动woker处理任务:```python manage.py qcluster```:

```
☁  django_q_demo [master] ⚡ python manage.py qcluster
08:26:20 [Q] INFO Q Cluster-26350 starting.
08:26:20 [Q] INFO Process-1:1 ready for work at 26353
08:26:20 [Q] INFO Process-1:2 ready for work at 26354
08:26:20 [Q] INFO Process-1:5 monitoring at 26357
08:26:20 [Q] INFO Process-1:4 ready for work at 26356
08:26:20 [Q] INFO Process-1:3 ready for work at 26355
08:26:20 [Q] INFO Process-1:6 pushing tasks at 26358
08:26:20 [Q] INFO Process-1 guarding cluster at 26352
08:26:20 [Q] INFO Q Cluster-26350 running.
```
可以看到创建了1个主进程、4个worker、1个monitoring、1个pushing、1个guarding




## Task

- 准备好后,就可以创建一些后台任务了. 看一下怎么创建一个task:

```python
from django_q.tasks import async, result
async(func, *args, hook=None, group=None, timeout=None, save=None, sync=False, cached=False, broker=None, q_options=None, **kwargs)
```
看起来参数挺多的,要是不小心函数使用了任务一些内置的关键字(如hook)等,这样会有意外的问题发生. `q_options`可以覆盖这些内置的关键字.所以建议的写法是将任务一些内置的关键字放到`q_options`里:
```python
opts = { 'task_name':'',
      'hook': 'hooks.print_result',
      'group': 'math',
      'timeout': 30
      }
task_id = async('math.modf', 2.5, q_options=opts)

task_result = result(task_id)

```

注意: **func不要传`task_name`这样的关键字参数,这样会被async取走的**.

任务结果获取:

```python
result(task_id, wait=0, cached=False)
```
`wait`是等待多少毫秒,`-1`表示无限等待. result函数是根据task的func返回值判断是否结束,**如果func没返回值,那么result认为任务一直没结束,这样在`wait=-1`时无限等待**.


- 可以使用`Async` 类去创建一个task,这样虽有的操作都在一个对象里了,更方便了:

```python
from django_q.tasks import Async
opts = { 'task_name':'fly',
      'group':'nsfocus',
      'timeout':30 }
a  = Async('math.floor', 1.5, q_options = opts)
a.run()
a.result( wait = 10)
```


## schedule

```python
schedule(func, *args, name=None, hook=None, schedule_type='O', minutes=None, repeats=-1, next_run=now(), q_options=None, **kwargs)
```

使用方法:
```python
schedule('django.core.management.call_command',
         'clearsessions',
         name='ggg',
         schedule_type='H',
         q_options={
             'task_name': 'xxx',
             'timeout': 60
         }
         )
```

schedule会根据配置去定期创建task的,name将作为task的group字段. 如果类型是MINUTES,需要指定参数minutes参数.
注意:
- name 被当做是唯一的，存在的话会报错.
- 当schedule_type是ONCE类型, 只会运行一次. 如果repeats <= 0的话，会被删除掉.

## 配置

如果是将django作为消息队列的话会进行如下配置:
```python
Q_CLUSTER = {
    'name': 'DjangORM',
    'workers': 4,
    'timeout': 90,
    'retry': 120,
    'queue_limit': 50,
    'bulk': 10,
    'orm': 'default'
}
```

- timeout:worker允许task执行的时长，可以针对单个任务设置. 默认是一直等待任务执行完毕.
- retry:中间件等待任务执行的时长,如果在这段时间任务还没执行完,那么会再次触发一个任务.
- save_limit: 控制成功的任务保存数量.  0表示不限制,-1表示不保存, 默认是250.失败的任务总是会保存起来的.
- queue_limit:控制进程队列的存储任务的个数,.主要是控制内存的占用.
- catch_up:字面意思是赶上进度, 比如当进程挂掉,会导致一段时间都没创建任务, 当进程恢复后, 是把漏掉的时间都补上还是一步跨越到未来即将执行的时间, 默认是True，即会追赶

这里timeout和retry之间有一些内在联系:
- retry是任务多久没执行完就再触发一个任务
- timeout是任务多久没执行完就杀掉

建议:
- 不设置timeout的话任务就没有超时限制了,可以针对单个任务设置
- 不设置retry的话默认是60秒,根据实际进行调整,因为要是有任务执行超过60秒的话会被再次触发.



## Chains
当需要顺序执行多个任务的时候

## 信号


任务即将入队和即将执行都会有信号发出,可以订阅这些信号执行一些动作m,例如
```python
from django.dispatch import receiver
from django_q.signals import pre_enqueue, pre_execute

@receiver(pre_enqueue)
def my_pre_enqueue_callback(sender, task, **kwargs):
    print("Task {} will be enqueued".format(task["name"]))

@receiver(pre_execute)
def my_pre_execute_callback(sender, func, task, **kwargs):
    print("Task {} will be executed by calling {}".format(
          task["name"], func))
```


## 为什么使用
为什么要使用django-q，我自己经常会有这样体会, 比如我在实现一个监控系统时，对主机内存、进程、数据库等监控都是会创建一个进程去执行, 很多项目可能都需要后台执行一些任务, 需要重复很多后台进程逻辑,很浪费.

## 例子